## Permütasyon Based Feature Importance

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Layer
import joblib
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences
import seaborn as sns
from scipy.stats import gaussian_kde
from sklearn.metrics import confusion_matrix, classification_report
import os
from datetime import datetime
import time

In [2]:
# Enable GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"Found {len(gpus)} GPU(s). GPU acceleration enabled.")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU found. Running on CPU.")

Found 1 GPU(s). GPU acceleration enabled.


In [3]:
# Create output directory for visualizations
output_dir = f"feature_importance_analysis_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
os.makedirs(output_dir, exist_ok=True)
print(f"Created output directory: {output_dir}")

Created output directory: feature_importance_analysis_20250620_232827


In [4]:
# Custom Self-Attention Layer definition (same as training)
class SelfAttention(Layer):
    def __init__(self, attention_units=128, return_attention=False, **kwargs):
        self.attention_units = attention_units
        self.return_attention = return_attention
        super(SelfAttention, self).__init__(**kwargs)
        
    def build(self, input_shape):
        self.time_steps = input_shape[1]
        self.input_dim = input_shape[2]
        
        self.query_dense = tf.keras.layers.Dense(self.attention_units)
        self.key_dense = tf.keras.layers.Dense(self.attention_units)
        self.value_dense = tf.keras.layers.Dense(self.input_dim)
        self.context_dense = tf.keras.layers.Dense(self.input_dim)
        
        super(SelfAttention, self).build(input_shape)
    
    def call(self, inputs):
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        
        score = tf.matmul(query, key, transpose_b=True)
        score = score / tf.math.sqrt(tf.cast(self.attention_units, tf.float32))
        
        attention_weights = tf.nn.softmax(score, axis=-1)
        context = tf.matmul(attention_weights, value)
        output = self.context_dense(context)
        
        if self.return_attention:
            return output, attention_weights
        return output
    
    def compute_output_shape(self, input_shape):
        if self.return_attention:
            return [(input_shape[0], input_shape[1], self.input_dim), 
                    (input_shape[0], input_shape[1], input_shape[1])]
        return (input_shape[0], input_shape[1], self.input_dim)
    
    def get_config(self):
        config = super(SelfAttention, self).get_config()
        config.update({
            'attention_units': self.attention_units,
            'return_attention': self.return_attention
        })
        return config

In [5]:
def custom_cost_metric(y_true, y_pred):
    y_pred = tf.nn.softmax(y_pred)
    pred_class = tf.argmax(y_pred, axis=1)
    true_class = tf.argmax(y_true, axis=1)
    
    cost_matrix = tf.constant([
        [0, 7, 8, 9, 10],
        [200, 0, 7, 8, 9],
        [300, 200, 0, 7, 8],
        [400, 300, 200, 0, 7],
        [500, 400, 300, 200, 0]
    ], dtype=tf.float32)
    
    costs = tf.gather_nd(cost_matrix, tf.stack([true_class, pred_class], axis=1))
    return tf.reduce_mean(costs)

In [6]:
model_dir = "trained_model_bilstm_attention_deneme_25_80296_27"

# Register custom objects
custom_objects = {
    'SelfAttention': SelfAttention,
    'custom_cost_metric': custom_cost_metric
}

# Load model
epoch_to_load = 27
model_path = f"{model_dir}/bilstm_attention_model_epoch_{epoch_to_load:02d}.keras"

print(f"Loading model from: {model_path}")
model = load_model(model_path, custom_objects=custom_objects)

Loading model from: trained_model_bilstm_attention_deneme_25_80296_27/bilstm_attention_model_epoch_27.keras


In [7]:
le = joblib.load(f"{model_dir}/label_encoder.joblib")
scaler = joblib.load(f"{model_dir}/standard_scaler.joblib")

In [9]:
validation_operational = pd.read_csv('selected_and_filled_validation_operational_readouts.csv')
validation_labels = pd.read_csv('validation_labels.csv')

# Get feature names
feature_names = validation_operational.columns[2:].tolist()

In [10]:
def prepare_all_data():
    """Prepare all validation data for analysis"""
    vehicle_groups = validation_operational.groupby('vehicle_id')
    vehicle_ids = list(vehicle_groups.groups.keys())
    
    X_data = []
    y_data = []
    vehicle_id_list = []
    
    print(f"Processing {len(vehicle_ids)} vehicles...")
    
    for i, vehicle_id in enumerate(vehicle_ids):
        if i % 100 == 0:
            print(f"Processing vehicle {i}/{len(vehicle_ids)}")
            
        group = vehicle_groups.get_group(vehicle_id)
        time_series = group.sort_values('time_step').iloc[:, 2:].values
        
        # Scale the data
        time_series_scaled = scaler.transform(time_series)
        
        # Get label
        label_row = validation_labels[validation_labels['vehicle_id'] == vehicle_id]
        if len(label_row) > 0:
            label = label_row['class_label'].values[0]
            encoded_label = le.transform([label])[0]
            
            X_data.append(time_series_scaled)
            y_data.append(encoded_label)
            vehicle_id_list.append(vehicle_id)
    
    # Pad sequences to same length
    max_length = max(len(seq) for seq in X_data)
    X_padded = pad_sequences(X_data, maxlen=max_length, padding='post', dtype='float32')
    
    print(f"Total samples prepared: {len(X_padded)}")
    print(f"Sequence length: {max_length}")
    print(f"Number of features: {X_padded.shape[2]}")
    
    return X_padded, np.array(y_data), vehicle_id_list

In [11]:
# GPU-accelerated permutation importance
@tf.function
def compute_predictions_gpu(model, inputs):
    """Compute predictions on GPU"""
    return model(inputs, training=False)

def compute_permutation_importance_gpu(model, X_data, y_data, batch_size=128, n_permutations=3, sample_size=None):
    """
    GPU-accelerated permutation importance computation
    """
    n_samples = len(X_data)
    n_features = X_data.shape[2]
    n_classes = len(le.classes_)
    
    if sample_size and sample_size < n_samples:
        indices = np.random.choice(n_samples, sample_size, replace=False)
        X_data = X_data[indices]
        y_data = y_data[indices]
        n_samples = sample_size
        print(f"Using {n_samples} samples for feature importance computation")
    
    # Store importance scores
    importance_scores = np.zeros((n_classes, n_features))
    
    print(f"\nComputing GPU-accelerated permutation importance...")
    print(f"Batch size: {batch_size}, Permutations: {n_permutations}")
    
    start_time = time.time()
    
    # Convert to TensorFlow dataset for efficient batching
    dataset = tf.data.Dataset.from_tensor_slices(X_data)
    dataset = dataset.batch(batch_size)
    
    # Get baseline predictions
    baseline_probs = []
    for batch in dataset:
        with tf.device('/GPU:0'):
            batch_pred = compute_predictions_gpu(model, batch)
            batch_prob = tf.nn.softmax(batch_pred)
            baseline_probs.append(batch_prob.numpy())
    baseline_probs = np.vstack(baseline_probs)
    
    # Process each feature
    for feature_idx in range(n_features):
        if feature_idx % 10 == 0:
            elapsed = time.time() - start_time
            print(f"Feature {feature_idx}/{n_features} - Elapsed: {elapsed:.1f}s")
        
        feature_importance = np.zeros(n_classes)
        
        # Run multiple permutations
        for perm in range(n_permutations):
            # Create permuted data
            X_permuted = X_data.copy()
            
            # Permute the feature
            for i in range(n_samples):
                perm_values = X_permuted[i, :, feature_idx].copy()
                np.random.shuffle(perm_values)
                X_permuted[i, :, feature_idx] = perm_values
            
            # Create dataset for permuted data
            perm_dataset = tf.data.Dataset.from_tensor_slices(X_permuted)
            perm_dataset = perm_dataset.batch(batch_size)
            
            # Get predictions with permuted feature
            permuted_probs = []
            for batch in perm_dataset:
                with tf.device('/GPU:0'):
                    batch_pred = compute_predictions_gpu(model, batch)
                    batch_prob = tf.nn.softmax(batch_pred)
                    permuted_probs.append(batch_prob.numpy())
            permuted_probs = np.vstack(permuted_probs)
            
            # Calculate importance for each class
            for class_idx in range(n_classes):
                importance = np.mean(np.abs(baseline_probs[:, class_idx] - permuted_probs[:, class_idx]))
                feature_importance[class_idx] += importance
        
        # Average over permutations
        importance_scores[:, feature_idx] = feature_importance / n_permutations
    
    total_time = time.time() - start_time
    print(f"Feature importance computation completed in {total_time:.1f} seconds")
    
    return importance_scores

In [12]:
print("Preparing all validation data for analysis...")
X_data, y_data, vehicle_ids = prepare_all_data()

# Check class distribution
unique_classes, class_counts = np.unique(y_data, return_counts=True)
print(f"\nClass distribution in data:")
for cls, count in zip(unique_classes, class_counts):
    class_label = le.inverse_transform([cls])[0]
    print(f"  Class {class_label}: {count} samples")

Preparing all validation data for analysis...
Processing 5046 vehicles...
Processing vehicle 0/5046
Processing vehicle 100/5046
Processing vehicle 200/5046
Processing vehicle 300/5046
Processing vehicle 400/5046
Processing vehicle 500/5046
Processing vehicle 600/5046
Processing vehicle 700/5046
Processing vehicle 800/5046
Processing vehicle 900/5046
Processing vehicle 1000/5046
Processing vehicle 1100/5046
Processing vehicle 1200/5046
Processing vehicle 1300/5046
Processing vehicle 1400/5046
Processing vehicle 1500/5046
Processing vehicle 1600/5046
Processing vehicle 1700/5046
Processing vehicle 1800/5046
Processing vehicle 1900/5046
Processing vehicle 2000/5046
Processing vehicle 2100/5046
Processing vehicle 2200/5046
Processing vehicle 2300/5046
Processing vehicle 2400/5046
Processing vehicle 2500/5046
Processing vehicle 2600/5046
Processing vehicle 2700/5046
Processing vehicle 2800/5046
Processing vehicle 2900/5046
Processing vehicle 3000/5046
Processing vehicle 3100/5046
Processing

In [13]:
# Compute feature importance using GPU acceleration
# Use sample_size to limit computation time
importance_scores = compute_permutation_importance_gpu(
    model, X_data, y_data, 
    batch_size=128, 
    n_permutations=3,  
    sample_size=min(2000, len(X_data))  # Use up to 2000 samples
)

Using 2000 samples for feature importance computation

Computing GPU-accelerated permutation importance...
Batch size: 128, Permutations: 3
Feature 0/61 - Elapsed: 7.0s
Feature 10/61 - Elapsed: 45.2s
Feature 20/61 - Elapsed: 84.1s
Feature 30/61 - Elapsed: 124.1s
Feature 40/61 - Elapsed: 166.5s
Feature 50/61 - Elapsed: 208.4s
Feature 60/61 - Elapsed: 251.6s
Feature importance computation completed in 255.9 seconds


In [14]:
# Convert to dictionary format
importance_dict = {}
for i in range(len(le.classes_)):
    if i in unique_classes:
        importance_dict[le.classes_[i]] = importance_scores[i]

In [15]:
print("\nCreating visualizations...")

# Visualization 1: Feature importance heatmap by class
plt.figure(figsize=(14, 8))
importance_df = pd.DataFrame(importance_dict, index=feature_names)
importance_df = importance_df.T

# Sort features by mean importance
mean_importance = importance_df.mean(axis=0)
top_features_idx = mean_importance.nlargest(30).index
importance_df_top = importance_df[top_features_idx]

sns.heatmap(importance_df_top, cmap='YlOrRd', cbar_kws={'label': 'Importance Score'}, 
            vmin=0, center=importance_df_top.mean().mean())
plt.title('Feature Importance Heatmap by Class (Top 30 Features)')
plt.xlabel('Features')
plt.ylabel('Classes')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'permutation_importance_heatmap.png'), dpi=300, bbox_inches='tight')
plt.close()


Creating visualizations...


In [16]:
# Visualization 2: Top features bar plot
plt.figure(figsize=(10, 8))
top_20_features = mean_importance.nlargest(20)
colors = plt.cm.viridis(np.linspace(0, 1, len(top_20_features)))
bars = plt.barh(top_20_features.index, top_20_features.values, color=colors)
plt.xlabel('Mean Importance Score')
plt.title('Top 20 Most Important Features (GPU-Accelerated Permutation)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'permutation_feature_importance_bar.png'), dpi=300, bbox_inches='tight')
plt.close()

In [17]:
# Visualization 3: Class-specific feature importance
n_classes_present = len(unique_classes)
fig, axes = plt.subplots((n_classes_present + 1) // 2, 2, figsize=(15, 4 * ((n_classes_present + 1) // 2)))
if n_classes_present == 1:
    axes = [axes]
else:
    axes = axes.flatten()

for idx, class_idx in enumerate(unique_classes):
    if idx < len(axes):
        ax = axes[idx]
        class_name = le.inverse_transform([class_idx])[0]
        if class_name in importance_df.index:
            class_importance = importance_df.loc[class_name].nlargest(15)
            colors = plt.cm.plasma(np.linspace(0, 1, len(class_importance)))
            ax.barh(class_importance.index, class_importance.values, color=colors)
            ax.set_xlabel('Importance Score')
            ax.set_title(f'Top 15 Features for Class: {class_name}')
            ax.grid(True, alpha=0.3)

# Hide extra subplots
for idx in range(n_classes_present, len(axes)):
    axes[idx].set_visible(False)

plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'class_specific_feature_importance.png'), dpi=300, bbox_inches='tight')
plt.close()

In [20]:
# Visualization 4: Feature importance variance
plt.figure(figsize=(12, 6))
feature_variance = importance_df.var(axis=0).nlargest(20)
plt.bar(range(len(feature_variance)), feature_variance.values, color='coral')
plt.xticks(range(len(feature_variance)), feature_variance.index, rotation=45, ha='right')
plt.xlabel('Features')
plt.ylabel('Variance in Importance')
plt.title('Features with Highest Variance in Importance Across Classes')
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'feature_importance_variance.png'), dpi=300, bbox_inches='tight')
plt.close()

# Save importance scores
importance_df.to_csv(os.path.join(output_dir, 'feature_importance_scores.csv'))

In [25]:
summary_stats = {
    'total_samples_analyzed': len(X_data),
    'samples_used_for_importance': min(2000, len(X_data)),
    'total_features': len(feature_names),
    'sequence_length': X_data.shape[1],
    'model_accuracy': np.mean(pred_classes == y_subset),
    'mean_confidence': np.mean(max_probs),
    'top_10_features': mean_importance.nlargest(10).to_dict(),
    'class_distribution': {str(le.inverse_transform([i])[0]): int(count) 
                          for i, count in zip(unique_classes, class_counts)}
}

In [26]:
print(f"\nTop 10 Most Important Features:")
for i, (feat, score) in enumerate(summary_stats['top_10_features'].items(), 1):
    print(f"{i:2d}. {feat}: {score:.4f}")


Top 10 Most Important Features:
 1. 397_24: 0.0224
 2. 397_0: 0.0148
 3. 397_8: 0.0143
 4. 397_9: 0.0138
 5. 397_25: 0.0126
 6. 397_11: 0.0119
 7. 291_5: 0.0112
 8. 397_30: 0.0104
 9. 459_2: 0.0103
10. 459_1: 0.0100
